In [ ]:
# Current directory: 
task_name = os.path.join('PUT LOCAL PATH HERE')

# Importing Libraries
import glob, os
import shutil
import cv2
import argparse
import time
import numpy as np
from tqdm import tqdm
from PIL import Image
from collections import defaultdict

### Check duplicated files in data folder
# Directory where the data will reside
path_data = task_name + 'data/'

# Get filenames with ext from directory and count each filenames without ext 
filenames = os.listdir(path_data)

D = defaultdict(list)
for name in filenames:
    D[name.split(".")[0]].append(name)

# Get filenames with ext which is not 2 pair 
nonpairfilenames = []
for key, value in D.items():
    if len(value) != 2:
        nonpairfilenames.append(value[0])

# Make "nonpairfiles" directory and move nonpairfilenames in it 
mov_path = task_name + "/nonpairfiles"
os.mkdir(mov_path)
for i in range(len(nonpairfilenames)):
    now_path = path_data + nonpairfilenames[i]
    shutil.move(now_path, mov_path)

### Randomly split into train and test sets, and move train sets to train directry
# Percentage of images to be used for the test set (default: 20% for test set)
percentage_test = 20;

# Create and/or truncate train.txt and test.txt
file_train = open(task_name + 'train.txt', 'w')  
file_test = open(task_name + 'test.txt', 'w')


# Populate train.txt and test.txt
counter = 1  
index_test = round(100 / percentage_test)  
trainfilename = []
traintextname = []
for pathAndFilename in glob.iglob(os.path.join(path_data, "*.jpg")):  
    title, ext = os.path.splitext(os.path.basename(pathAndFilename))

    if counter == index_test:
        counter = 1
        file_test.write(path_data + title + '.jpg' + "\n")
    else:
        trainfilename.append(pathAndFilename)
        file_train.write(path_data + title + '.jpg' + "\n")
        counter = counter + 1
file_train.close()
file_test.close()

# Read train.txt each line and write them in traintextname
with open(task_name + 'train.txt', 'r') as f:
    for line in f:
        traintextname.append(line)

# Change .jpg to .txt in traintextname
traintextname = [s.replace('.jpg', '.txt').replace('\n', '') for s in traintextname]

# Directory where the train data will reside
path_train = task_name + 'train/'
os.mkdir(path_train)

for i in range(len(trainfilename)):
    new_path_file = shutil.move(trainfilename[i], path_train)
    new_path_text = shutil.move(traintextname[i], path_train)
   
    
###convert from Yolo_mark to opencv format
def yoloFormattocv(x1, y1, x2, y2, H, W):
    bbox_width = x2 * W
    bbox_height = y2 * H
    center_x = x1 * W
    center_y = y1 * H
    voc = []
    voc.append(center_x - (bbox_width / 2))
    voc.append(center_y - (bbox_height / 2))
    voc.append(center_x + (bbox_width / 2))
    voc.append(center_y + (bbox_height / 2))
    return [int(v) for v in voc]

# Convert from opencv format to yolo format
# H,W is the image height and width
def cvFormattoYolo(corner, H, W):
    bbox_W = corner[3] - corner[1]
    bbox_H = corner[4] - corner[2]
    center_bbox_x = (corner[1] + corner[3]) / 2
    center_bbox_y = (corner[2] + corner[4]) / 2
    return corner[0], round(center_bbox_x / W, 6), round(center_bbox_y / H, 6), round(bbox_W / W, 6), round(bbox_H / H, 6)

class yoloRotatebbox:
    def __init__(self, filename, image_ext, angle):
        assert os.path.isfile(filename + image_ext)
        assert os.path.isfile(filename + '.txt')
        
        self.filename = filename
        self.image_ext = image_ext
        self.angle = angle
        
        # Read image using cv2
        self.image = cv2.imread(self.filename + self.image_ext, 1)
        
        rotation_angle = self.angle * np.pi / 180
        self.rot_matrix = np.array(
            [[np.cos(rotation_angle), -np.sin(rotation_angle)], [np.sin(rotation_angle), np.cos(rotation_angle)]])
    def rotateYolobbox(self):
        new_height, new_width = self.rotate_image().shape[:2]
        f = open(self.filename + '.txt', 'r')
        f1 = f.readlines()
        new_bbox = []
        H, W = self.image.shape[:2]
        for x in f1:
            bbox = x.strip('\n').split(' ')
            if len(bbox) > 1:
                (center_x, center_y, bbox_width, bbox_height) = yoloFormattocv(float(bbox[1]), float(bbox[2]),
                                                                               float(bbox[3]), float(bbox[4]), H, W)
                upper_left_corner_shift = (center_x - W / 2, -H / 2 + center_y)
                upper_right_corner_shift = (bbox_width - W / 2, -H / 2 + center_y)
                lower_left_corner_shift = (center_x - W / 2, -H / 2 + bbox_height)
                lower_right_corner_shift = (bbox_width - W / 2, -H / 2 + bbox_height)
                new_lower_right_corner = [-1, -1]
                new_upper_left_corner = []
                for i in (upper_left_corner_shift, upper_right_corner_shift, lower_left_corner_shift,
                          lower_right_corner_shift):
                    new_coords = np.matmul(self.rot_matrix, np.array((i[0], -i[1])))
                    x_prime, y_prime = new_width / 2 + new_coords[0], new_height / 2 - new_coords[1]
                    if new_lower_right_corner[0] < x_prime:
                        new_lower_right_corner[0] = x_prime
                    if new_lower_right_corner[1] < y_prime:
                        new_lower_right_corner[1] = y_prime
                    if len(new_upper_left_corner) > 0:
                        if new_upper_left_corner[0] > x_prime:
                            new_upper_left_corner[0] = x_prime
                        if new_upper_left_corner[1] > y_prime:
                            new_upper_left_corner[1] = y_prime
                    else:
                        new_upper_left_corner.append(x_prime)
                        new_upper_left_corner.append(y_prime)
                new_bbox.append([bbox[0], new_upper_left_corner[0], new_upper_left_corner[1],
                                 new_lower_right_corner[0], new_lower_right_corner[1]])
        return new_bbox
    def rotate_image(self):
        """
        Rotates an image (angle in degrees) and expands image to avoid cropping
        """
        height, width = self.image.shape[:2]  # image shape has 3 dimensions
        image_center = (width / 2,
                        height / 2)  # getRotationMatrix2D needs coordinates in reverse order (width, height) compared to shape
        rotation_mat = cv2.getRotationMatrix2D(image_center, self.angle, 1.)
        # rotation calculates the cos and sin, taking absolutes of those.
        abs_cos = abs(rotation_mat[0, 0])
        abs_sin = abs(rotation_mat[0, 1])
        # find the new width and height bounds
        bound_w = int(height * abs_sin + width * abs_cos)
        bound_h = int(height * abs_cos + width * abs_sin)
        # subtract old image center (bringing image back to origin) and adding the new image center coordinates
        rotation_mat[0, 2] += bound_w / 2 - image_center[0]
        rotation_mat[1, 2] += bound_h / 2 - image_center[1]
        # rotate image with the new bounds and translated rotation matrix
        rotated_mat = cv2.warpAffine(self.image, rotation_mat, (bound_w, bound_h))
        return rotated_mat

### Put angles for rotating images (default: 90, 180, and 270)
os.chdir(path_train)
if __name__ == "__main__":
    angels=[90,180,270]
    for filename in tqdm(os.listdir()):
        file =filename.split(".")
        print(file)
        if(file[-1]=="jpg"):
            image_name=file[0]
            image_ext="."+file[1]
        else:
            continue
        for angle in angels:
            im = yoloRotatebbox(image_name, image_ext, angle)
            bbox = im.rotateYolobbox()
            image = im.rotate_image()
            # to write rotateed image to disk
            cv2.imwrite(image_name+'_' + str(angle) + '.jpg', image)
            file_name = image_name+'_' + str(angle) + '.txt'
            #print("For angle "+str(angle))
            if os.path.exists(file_name):
                os.remove(file_name)
            # to write the new rotated bboxes to file
            for i in bbox:
                with open(file_name, 'a') as fout:
                    fout.writelines(
                        ' '.join(map(str, cvFormattoYolo(i, im.rotate_image().shape[0], im.rotate_image().shape[1]))) + '\n')    

In [ ]:
# Create train2.txt that includes augmentated data file list
# This file will be uploaded and rename as train.txt; then, YOLO can use this file for training.

path_train = task_name + 'train/'
file_train2 = open(task_name + 'train2.txt', 'w')  

# Populate train.txt and test.txt
trainfilename2 = []

for pathAndFilename in glob.iglob(os.path.join(path_train, "*.jpg")):  
    title, ext = os.path.splitext(os.path.basename(pathAndFilename))
    trainfilename2.append(pathAndFilename)
    file_train2.write('/content/darknet/task1/data/' + title + '.jpg' + "\n")
file_train2.close()

# Create test2.txt 
# This file will be uploaded and rename as test.txt; then, YOLO can use this file for training.

file_test2 = open(task_name + 'test2.txt', 'w')  

# Populate train.txt and test.txt
testfilename2 = []

for pathAndFilename in glob.iglob(os.path.join(path_data, "*.jpg")):  
    title, ext = os.path.splitext(os.path.basename(pathAndFilename))
    testfilename2.append(pathAndFilename)
    file_test2.write('/content/darknet/task1/data/' + title + '.jpg' + "\n")
        
file_test2.close()